In [12]:
import os
import dill
import numpy as np
import jax.numpy as jnp
import geopandas as gpd
import numpyro
from numpyro.infer import Predictive
import arviz as az

import sys
sys.path.append(os.path.pardir)

from aggGP import case_est_model_gp_aggr, exp_sq_kernel, M_g

In [13]:
# Load variables
# --------------------------------- Load Data -------------------------------- #
data_dir = "../data/processed"

# Lat/Lon Values of artificial grid
x = np.load(os.path.join(data_dir,"lat_lon_x.npy")) #(2618,2)
# Low regional data
pol_pt_lo = np.load(os.path.join(data_dir,"low","pol_pt_lo.npy")) #(9,2618)
pt_which_pol_lo = np.load(os.path.join(data_dir,"low","pt_which_pol_lo.npy")) #(2618,)
# High regional data 
pol_pt_hi = np.load(os.path.join(data_dir,"high","pol_pt_hi.npy")) #(49, 2618,)
pt_which_pol_hi = np.load(os.path.join(data_dir,"high","pt_which_pol_hi.npy")) #(2618,)
# Dataframes
df_lo = gpd.read_file(os.path.join(data_dir, "low", "us_census_divisions","us_census_divisions.shp"))
df_hi = gpd.read_file(os.path.join(data_dir, "high", "us_state_divisions","us_state_divisions.shp"))


In [14]:
# Arguments
args = {
    #"n_low_obs" : df_lo.tot_cases, #observarions <- This is passed as y
    "n_low_specs" : jnp.array(df_lo.tot_specs),  
    "x" : jnp.array(x),
    "gp_kernel" : exp_sq_kernel,
    "jitter" : 1e-4,
    "noise" : 1e-4,
    "M" : pol_pt_hi,
    #todo : You can probably remove M2 as this is not needed
    "M2" : pol_pt_hi,
    "combine" : False # Get combined results for 
}

In [17]:
# Load MCMC 
with open("../model_weights/aggGP_samples1000_tt14013.68963599205s") as file:
    mcmc = dill.load(file)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
pos_samples = mcmc.get_samples()
mcmc.print_summary(exclude_deterministic = False)

ss = numpyro.diagnostics.summary(mcmc.get_samples(group_by_chain = True))
r = np.mean(ss["gp_aggr"])
print(f"Average ESS for all aggGP effects : {round(r)}")

print(f"kernel_length R-hat : {round(ss['kernel_length']['r_hat'], 2)}")
print(f"kernel_var R-hat : {round(ss['kernel_var']['r_hat'],2)}")


In [1]:
case_samples_az_gp = az.from_numpyro(mcmc)
az.plot_trace(case_samples_az_gp, var_names = ("kernel_length"))
az.plot_trace(case_samples_az_gp, var_names = ("kernel_var"))

NameError: name 'end' is not defined

In [ ]:
# Posterior predictions
case_pos = Predictive(prev_model_gp_aggr, pos_samples)(random.PRNGKey(1), args)

